#Abstractions in decision making
TODO: intro

## Taxonomy example
TODO: short description

In [1]:
using RateDistortionDecisionMaking, Gadfly

#make the default plot size a bit larger
set_default_plot_size(15cm, 12cm)


#set up taxonomy example
include("TaxonomyExample.jl")
w_vec, w_strings, a_vec, a_strings, p_w, U = setuptaxonomy()


#pre-compute utilities, find maxima
U_pre, Umax = setuputilityarrays(a_vec, w_vec,U)

#visualize utility
plt_utility = visualizeMatrix(U_pre, w_vec, a_vec, w_strings, a_strings, xlabel="Item bought w",
                              ylabel="Recommend a", legendlabel="U(a,w)")


LoadError: type cannot be constructed
while loading In[1], in expression starting on line 16

### Parameters
Set the inverse temperature below for computing a bounded rational solution using Blahut-Arimoto iterations. Make sure to re-evaluate the subsequent cells accordingly.

In [2]:
β = 1.3 #inverse temperature

1.3

In [3]:
ε = 0.00001 #convergence critetion for BAiterations
maxiter = 10000 #maximum number of BA iterations

#initialize p(a) uniformly
num_acts = length(a_vec)
pa_init = ones(num_acts)/num_acts 

#Blahut-Arimotot iterations
p_agw, p_a, performance = BAiterations(pa_init, β, U_pre, p_w, ε, maxiter, compute_performance=true, 
                                       performance_as_dataframe=true, performance_per_iteration=false)


#visualize solution
#suppress immediate plotting since the stacked plots look bad, rather use display() to show each plot separately
plot_marg, plot_cond = visualizeBAsolution(p_a, p_agw, a_vec, w_vec, a_strings, w_strings,
                                           wlabel="Item bought w", alabel="Recommend a",
                                           legendlabel_marginal="p(a)", legendlabel_conditional="p*(a|w)",suppress_vis=true);
display(plot_marg)
display(plot_cond)


LoadError: type cannot be constructed
while loading In[3], in expression starting on line 15

## Sweep over $\beta$ values to see the different levels of abstraction emerge

Note that the code below might take a few minutes to run - you can speed it up by using a lower-resolution $\beta$-grid: ``β_sweep``

In [ ]:
#β-sweep
ε = 0.00001 #convergence critetion for BAiterations
maxiter = 20000
β_sweep = [0.01:0.005:3]
#β_sweep = [0.1:0.05:2, 2:0.5:15]

nβ = length(β_sweep)

#preallocate
I = zeros(nβ)
Ha = zeros(nβ)
Hagw = zeros(nβ)
EU = zeros(nβ)
RDobj = zeros(nβ)

#sweep over β values
for i=1:nβ    
    pagw, pa, I[i], Ha[i], Hagw[i], EU[i], RDobj[i] = BAiterations(pa_init, β_sweep[i], U_pre,  p_w, ε, maxiter,compute_performance=true)  
end

#plot evolution of performance measures
plot_perf_entropy, plot_perf_util, plot_rateutility = plotperformancemeasures(I, Ha, Hagw, EU, RDobj, β_sweep);

In [ ]:
#create a plot where the entropic performance plot is shown and on top of it several solutions of p(a|w)
#for different tempereatures (ideally lying in the plateaus)

β_plts = [0.05, 0.5, 0.9, 1.1, 1.3, 1.5, 2]

nβ = length(β_plts)

#preallocate
I = zeros(nβ)
Ha = zeros(nβ)
Hagw = zeros(nβ)
EU = zeros(nβ)
RDobj = zeros(nβ)
plots = Array(Plot,nβ)

for i=1:nβ    
    pagw, pa, I[i], Ha[i], Hagw[i], EU[i], RDobj[i] = BAiterations(pa_init, β_plts[i], U_pre, p_w, ε, maxiter,compute_performance=true)  

    #except for the last plot, provide an optional argument that will be passed on to the underlying Gadfly Theme()
    #to suppress drawing of the colorkey (legend)
    β_val = β_plts[i]
    if i==1
        pcond = visualizeBAconditional(pagw, a_vec, w_vec, wlabel="Item bought w β=$β_val", alabel="Recommend a", 
        legendlabel="", key_position = :none, minor_label_font_size = 0pt,  major_label_font_size = 9pt,
        bar_spacing = 0pt)
    elseif i<nβ
        pcond = visualizeBAconditional(pagw, a_vec, w_vec, wlabel="β=$β_val", alabel="",
        legendlabel="", key_position = :none,  minor_label_font_size = 0pt,  major_label_font_size = 9pt,
        bar_spacing = 0pt)
    else
        pcond = visualizeBAconditional(pagw, a_vec, w_vec, wlabel="β=$β_val", alabel="",
        legendlabel="p*(a|w)",  minor_label_font_size = 0pt, major_label_font_size = 9pt,
        key_title_font_size = 9pt, key_label_font_size = 8pt, bar_spacing = 0pt )
    end

    #display(pcond)
    plots[i] = pcond
end

#compose final plot
plot_evolution = vstack(hstack(plots...),plot_perf_entropy)
display(plot_evolution)

In [ ]:
#uncomment to store plots

plot_setup = hstack(plt_utility, plot_cond)
#display(plot_setup)
#draw(SVG("Figures/TaskSetupAndExmplSolution_Taxonomy.svg",22cm,12cm),plot_setup)


plot_evolution_paper = vstack(plot_perf_util, plot_perf_entropy)
plot_solutions = hstack(plots...)
#display(plot_evolution_paper)
#display(plot_solutions)
#draw(SVG("Figures/TempSweepEvolution.svg", 18cm, 14cm), plot_evolution_paper)
#draw(SVG("Figures/TempSweepSolutions.svg", 18cm, 7cm), plot_solutions)

#display(plot_rateutility)
#draw(SVG("Figures/RateUtilityCurve.svg", 8.5cm, 7cm), plot_rateutility)